In [2]:
import pandas as pd
import numpy as np 
from collections import Counter
from MarkovChain import MarkovChain
import splitter
import tldextract
from scipy.sparse import csr_matrix

In [3]:
alexa_df = pd.read_csv('resources/top-1m.csv', names=['rank', 'domain'])

In [ ]:
alexa_df = pd.DataFrame.from_dict([
    {'domain': 'helloworld.com'},
    {'domain': 'helloninja.com'},
    {'domain': 'lovecats.com'},
    {'domain': 'lovedogs.com'},
    {'domain': 'lovewomen.com'},
    {'domain': 'ninjawomen.com'},
    {'domain': 'ninjakid.com'},
    {'domain': 'ninjaboy.com'},
    {'domain': 'ninjamen.com'},
])

In [4]:
%%time
round_ndigits = 8
words_transitions_count = {}
states_set = set()


for domain in list(alexa_df['domain'][:1000]):
    exr = tldextract.extract(domain)
    words = splitter.split(exr.domain.replace('-','').replace('2', 'to').replace('4', 'for'))
    if type(words) == list and len(words) > 1:
        next_word = words[-1]
        for word in reversed(words[:-1]):
            if word not in words_transitions_count:
                words_transitions_count[word] = Counter()
                words_transitions_count[word][word] = 0
            words_transitions_count[word].update({next_word: 1})
            states_set.add(word)
            states_set.add(next_word)

words_transitions_prob = {}
for word in words_transitions_count.keys():
    words_transitions_prob[word] = {}
    word_sum = sum(words_transitions_count[word].values())
    for next_word, val in words_transitions_count[word].items():
        words_transitions_prob[word][next_word] = round(val / word_sum, round_ndigits)
    row_sum = round(sum(words_transitions_prob[word].values()), round_ndigits)
    
    if row_sum != 1:
#         print(f'word: {word}, row_sum: {row_sum}')
        error = round((row_sum - 1) * (10 ** round_ndigits))
        sign = -1 if error > 0 else 1
        adj_count = np.abs(error)
#         print (f'sign: {sign}, adj_count: {adj_count}')
        for key in words_transitions_prob[word]:
#             print('pre fix:', words_transitions_prob[word][key])
            words_transitions_prob[word][key] += sign * (10 ** (-1 * round_ndigits))
#             print('post fix:', words_transitions_prob[word][key])
            adj_count -= 1
            if adj_count == 0:
                break

for state in states_set:
    if state not in words_transitions_prob:
        words_transitions_prob[state] = {}
                
df = pd.DataFrame.from_dict(words_transitions_prob).T
df.fillna(0, inplace=True)
transition_matrix = df[df.index].to_numpy()

Wall time: 50.7 s


In [5]:
name_chain = MarkovChain(transition_matrix=transition_matrix, states=list(df.index))

In [6]:
# name_chain = MarkovChain(transition_matrix=transition_matrix, states=['cats', 'dogs', 'love', 'ninja', 'women', 'world'])

name_chain.next_state(current_state='hello')

KeyError: 'hello'

In [8]:
for i in range(10):
    print (name_chain.next_state('you'))

Ku
tube
Ku
tube
Ku
Ku
porn
Ku
Ku
porn


In [ ]:
for i in range(transition_matrix.shape[0]):
    print(transition_matrix[i])
    transition_matrix[i] /= transition_matrix[i].sum()
    print(transition_matrix[i])
    print('----------------------------------------------------')

In [ ]:
name_chain = MarkovChain(transition_matrix=transition_matrix, states=list(words_transitions_prob.keys()))

In [ ]:
transition_matrix.shape

In [ ]:
len(list(words_transitions_prob.keys()))

In [ ]:
name_chain.next_state(current_state='you')

In [ ]:
for val in words_transitions_prob['the'].values():
    print (val)

In [ ]:
sum(words_transitions_prob['the'].values())

In [ ]:
for key, trans in words_transitions_prob.items():
    print (key, sum(trans.values()))

In [ ]:
transition_matrix

In [ ]:
sign

In [ ]:
words_transitions_prob['you']

In [ ]:
d= {'a': 2, 'b': 4}

In [ ]:
for key in d:
    print (key)

In [ ]:
10 ** -1 * round_ndigits

In [ ]:
np.power(10, -6)

In [ ]:
sum(words_transitions_prob['porn'].values())

In [ ]:
round((1.0002 - 1) * 10000)

In [ ]:
df = pd.DataFrame.from_dict(words_transitions_prob)
df.fillna(0, inplace=True)
transition_matrix = df.to_numpy()

In [ ]:
sum(df['you'])

In [ ]:
list(words_transitions_prob.keys())

In [ ]:
transition_matrix.shape

In [ ]:
name_chain = MarkovChain(transition_matrix=transition_matrix, states=list(words_transitions_prob.keys()))
name_chain.next_state(current_state='hello')
# predictions = []
# for i in range(10000):
#     y = weather_chain.next_state(current_state='Sunny')
#     y = weather_chain.next_state(current_state=y)
#     predictions.append(y)

# # predictions = weather_chain.generate_states(current_state='Rainy', no=10000)
# Counter(predictions)

In [ ]:
df = pd.DataFrame.from_dict(words_transitions)

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
d = Counter({'a': 2, 'b': 5, 'c': 1, 'd': 2})

In [ ]:
d

In [ ]:
row_sum = sum(d.values())

In [ ]:
for key, val in d.items():
    d[key] = val / row_sum

In [ ]:
d['a']

In [ ]:
df[df['porn'] > 0]